# Ray Path Visualization

This notebook demonstrates how to extract and visualize seismic ray paths through 1D Earth models using the `seisray` package.

## Learning Objectives
- Extract ray paths for different seismic phases
- Visualize ray paths on circular Earth cross-sections
- Analyze ray path properties (turning points, pierce points)
- Compare ray paths between different phases and models

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import sys
import os

# Add the parent directory to the path to import seisray
sys.path.append(os.path.dirname(os.getcwd()))

from seisray import RayPathTracer, EarthPlotter

print("Successfully imported seisray package!")

## 1. Basic Ray Path Extraction

Let's start by extracting ray paths for P and S waves at a specific distance.

In [ ]:
# Set up parameters
source_depth = 10  # km
distance = 60      # degrees

# Create ray path tracer
tracer = RayPathTracer('iasp91')

# Extract P-wave ray path
p_rays = tracer.get_ray_paths(source_depth, distance, phases=['P'])
print(f"Found {len(p_rays)} P-wave ray path(s)")

# Extract S-wave ray path
s_rays = tracer.get_ray_paths(source_depth, distance, phases=['S'])
print(f"Found {len(s_rays)} S-wave ray path(s)")

# Analyze the P-wave ray path
if p_rays:
    p_ray = p_rays[0]
    print(f"\nP-wave ray path details:")
    print(f"  Number of points: {len(p_ray.path['dist'])}")
    print(f"  Maximum depth: {np.max(p_ray.path['depth']):.1f} km")
    print(f"  Travel time: {p_ray.time:.1f} s")
    print(f"  Phase name: {p_ray.name}")

## 2. Visualizing Ray Paths on Earth Cross-Section

Now let's create a circular Earth cross-section and plot the ray paths.

In [ ]:
# Create Earth plotter
plotter = EarthPlotter()

# Plot single ray path
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# Plot Earth circle
earth_circle = Circle((0, 0), 6371, fill=False, color='black', linewidth=2)
ax.add_patch(earth_circle)

# Plot core-mantle boundary
cmb_circle = Circle((0, 0), 3480, fill=False, color='gray', linewidth=1, linestyle='--')
ax.add_patch(cmb_circle)

# Convert ray path to Cartesian coordinates for plotting
if p_rays:
    ray = p_rays[0]
    distances_rad = np.radians(ray.path['dist'])
    radii = 6371 - ray.path['depth']

    x = radii * np.sin(distances_rad)
    y = radii * np.cos(distances_rad)

    ax.plot(x, y, 'b-', linewidth=2, label='P-wave')

# Plot source and receiver
source_x = (6371 - source_depth) * np.sin(0)
source_y = (6371 - source_depth) * np.cos(0)
ax.plot(source_x, source_y, 'r*', markersize=15, label='Source')

receiver_x = 6371 * np.sin(np.radians(distance))
receiver_y = 6371 * np.cos(np.radians(distance))
ax.plot(receiver_x, receiver_y, 'g^', markersize=10, label='Receiver')

ax.set_xlim(-7000, 7000)
ax.set_ylim(-7000, 7000)
ax.set_aspect('equal')
ax.set_xlabel('Distance (km)')
ax.set_ylabel('Distance (km)')
ax.set_title(f'P-wave Ray Path (Distance = {distance}°, Depth = {source_depth} km)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.show()

## 3. Comparing P and S Wave Ray Paths

Let's plot both P and S wave ray paths on the same diagram.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 10))

# Plot Earth structure
earth_circle = Circle((0, 0), 6371, fill=False, color='black', linewidth=2)
ax.add_patch(earth_circle)

cmb_circle = Circle((0, 0), 3480, fill=False, color='gray', linewidth=1, linestyle='--')
ax.add_patch(cmb_circle)

# Plot P-wave ray path
if p_rays:
    ray = p_rays[0]
    distances_rad = np.radians(ray.path['dist'])
    radii = 6371 - ray.path['depth']

    x = radii * np.sin(distances_rad)
    y = radii * np.cos(distances_rad)

    ax.plot(x, y, 'b-', linewidth=3, label=f'P-wave ({ray.time:.1f} s)')

# Plot S-wave ray path
if s_rays:
    ray = s_rays[0]
    distances_rad = np.radians(ray.path['dist'])
    radii = 6371 - ray.path['depth']

    x = radii * np.sin(distances_rad)
    y = radii * np.cos(distances_rad)

    ax.plot(x, y, 'r-', linewidth=3, label=f'S-wave ({ray.time:.1f} s)')

# Plot source and receiver
source_x = (6371 - source_depth) * np.sin(0)
source_y = (6371 - source_depth) * np.cos(0)
ax.plot(source_x, source_y, 'k*', markersize=15, label='Source')

receiver_x = 6371 * np.sin(np.radians(distance))
receiver_y = 6371 * np.cos(np.radians(distance))
ax.plot(receiver_x, receiver_y, 'k^', markersize=12, label='Receiver')

ax.set_xlim(-7000, 7000)
ax.set_ylim(-7000, 7000)
ax.set_aspect('equal')
ax.set_xlabel('Distance (km)')
ax.set_ylabel('Distance (km)')
ax.set_title(f'P and S Wave Ray Paths (Distance = {distance}°, Depth = {source_depth} km)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.show()

## 4. Ray Paths at Different Distances

Let's visualize how P-wave ray paths change with epicentral distance.

In [ ]:
# Different distances to analyze
distances = [20, 40, 60, 80]
colors = ['blue', 'green', 'red', 'purple']

fig, ax = plt.subplots(1, 1, figsize=(12, 10))

# Plot Earth structure
earth_circle = Circle((0, 0), 6371, fill=False, color='black', linewidth=2)
ax.add_patch(earth_circle)

cmb_circle = Circle((0, 0), 3480, fill=False, color='gray', linewidth=1, linestyle='--')
ax.add_patch(cmb_circle)

# Plot ray paths for different distances
for distance, color in zip(distances, colors):
    rays = tracer.get_ray_paths(source_depth, distance, phases=['P'])

    if rays:
        ray = rays[0]
        distances_rad = np.radians(ray.path['dist'])
        radii = 6371 - ray.path['depth']

        x = radii * np.sin(distances_rad)
        y = radii * np.cos(distances_rad)

        ax.plot(x, y, color=color, linewidth=2,
               label=f'{distance}° ({ray.time:.1f} s, max depth: {np.max(ray.path["depth"]):.0f} km)')

        # Plot receiver position
        receiver_x = 6371 * np.sin(np.radians(distance))
        receiver_y = 6371 * np.cos(np.radians(distance))
        ax.plot(receiver_x, receiver_y, 'o', color=color, markersize=8)

# Plot source
source_x = (6371 - source_depth) * np.sin(0)
source_y = (6371 - source_depth) * np.cos(0)
ax.plot(source_x, source_y, 'k*', markersize=15, label='Source')

ax.set_xlim(-7000, 7000)
ax.set_ylim(-1000, 7000)
ax.set_aspect('equal')
ax.set_xlabel('Distance (km)')
ax.set_ylabel('Distance (km)')
ax.set_title(f'P-wave Ray Paths at Different Distances (Depth = {source_depth} km)')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Analyzing Ray Path Properties

Let's extract and analyze specific properties of the ray paths.

In [ ]:
# Analyze ray paths at different distances
distances = np.linspace(10, 90, 17)
max_depths = []
travel_times = []
turning_depths = []

print(f"{'Distance (°)':<12} {'Travel Time (s)':<15} {'Max Depth (km)':<15} {'Turning Depth (km)':<15}")
print("-" * 70)

for distance in distances:
    rays = tracer.get_ray_paths(source_depth, distance, phases=['P'])

    if rays:
        ray = rays[0]
        max_depth = np.max(ray.path['depth'])
        travel_time = ray.time

        # Find turning point (deepest point)
        turning_depth = max_depth

        max_depths.append(max_depth)
        travel_times.append(travel_time)
        turning_depths.append(turning_depth)

        print(f"{distance:<12.1f} {travel_time:<15.1f} {max_depth:<15.1f} {turning_depth:<15.1f}")
    else:
        max_depths.append(np.nan)
        travel_times.append(np.nan)
        turning_depths.append(np.nan)
        print(f"{distance:<12.1f} {'No ray found':<15} {'N/A':<15} {'N/A':<15}")

In [ ]:
# Plot ray path properties
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Travel time vs distance
axes[0, 0].plot(distances, np.array(travel_times)/60, 'bo-', linewidth=2, markersize=6)
axes[0, 0].set_xlabel('Distance (degrees)')
axes[0, 0].set_ylabel('Travel Time (minutes)')
axes[0, 0].set_title('P-wave Travel Time vs Distance')
axes[0, 0].grid(True, alpha=0.3)

# Maximum depth vs distance
axes[0, 1].plot(distances, max_depths, 'ro-', linewidth=2, markersize=6)
axes[0, 1].set_xlabel('Distance (degrees)')
axes[0, 1].set_ylabel('Maximum Depth (km)')
axes[0, 1].set_title('Ray Path Maximum Depth vs Distance')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].invert_yaxis()

# Travel time vs max depth
axes[1, 0].plot(max_depths, np.array(travel_times)/60, 'go-', linewidth=2, markersize=6)
axes[1, 0].set_xlabel('Maximum Depth (km)')
axes[1, 0].set_ylabel('Travel Time (minutes)')
axes[1, 0].set_title('Travel Time vs Maximum Depth')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].invert_xaxis()

# Ray parameter analysis
ray_parameters = []
for distance in distances:
    rays = tracer.get_ray_paths(source_depth, distance, phases=['P'])
    if rays:
        ray_parameters.append(rays[0].ray_param)
    else:
        ray_parameters.append(np.nan)

axes[1, 1].plot(distances, ray_parameters, 'mo-', linewidth=2, markersize=6)
axes[1, 1].set_xlabel('Distance (degrees)')
axes[1, 1].set_ylabel('Ray Parameter (s/degree)')
axes[1, 1].set_title('Ray Parameter vs Distance')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Multiple Phases Comparison

Let's look at different seismic phases and their ray paths.

In [ ]:
# Get multiple phases for a specific distance
distance = 40
phases = ['P', 'S', 'PP', 'SS']
colors = ['blue', 'red', 'cyan', 'magenta']

fig, ax = plt.subplots(1, 1, figsize=(12, 10))

# Plot Earth structure
earth_circle = Circle((0, 0), 6371, fill=False, color='black', linewidth=2)
ax.add_patch(earth_circle)

cmb_circle = Circle((0, 0), 3480, fill=False, color='gray', linewidth=1, linestyle='--')
ax.add_patch(cmb_circle)

print(f"Ray paths for distance = {distance}°:")
print("-" * 40)

for phase, color in zip(phases, colors):
    rays = tracer.get_ray_paths(source_depth, distance, phases=[phase])

    if rays:
        ray = rays[0]
        distances_rad = np.radians(ray.path['dist'])
        radii = 6371 - ray.path['depth']

        x = radii * np.sin(distances_rad)
        y = radii * np.cos(distances_rad)

        ax.plot(x, y, color=color, linewidth=2,
               label=f'{phase} ({ray.time:.1f} s)')

        print(f"{phase:<4} Travel time: {ray.time:.1f} s, Max depth: {np.max(ray.path['depth']):.1f} km")
    else:
        print(f"{phase:<4} No ray found")

# Plot source and receiver
source_x = (6371 - source_depth) * np.sin(0)
source_y = (6371 - source_depth) * np.cos(0)
ax.plot(source_x, source_y, 'k*', markersize=15, label='Source')

receiver_x = 6371 * np.sin(np.radians(distance))
receiver_y = 6371 * np.cos(np.radians(distance))
ax.plot(receiver_x, receiver_y, 'k^', markersize=12, label='Receiver')

ax.set_xlim(-7000, 7000)
ax.set_ylim(-7000, 7000)
ax.set_aspect('equal')
ax.set_xlabel('Distance (km)')
ax.set_ylabel('Distance (km)')
ax.set_title(f'Multiple Phase Ray Paths (Distance = {distance}°, Depth = {source_depth} km)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.show()

## Summary

In this notebook, we demonstrated:

1. **Ray path extraction** using the `RayPathTracer` class
2. **Circular Earth visualization** with ray paths plotted on cross-sections
3. **P and S wave comparison** showing different ray path geometries
4. **Distance effects** on ray path geometry and travel times
5. **Ray path analysis** including maximum depths and ray parameters
6. **Multiple phase comparison** showing different seismic phases

### Key Observations:
- Ray paths curve due to velocity increase with depth
- Longer distances correspond to deeper ray paths
- S-waves follow similar paths to P-waves but are slower
- Multiple phases (PP, SS) show more complex ray geometries
- Ray parameter decreases with increasing distance
- Maximum depth increases approximately linearly with distance for P-waves

The `seisray` package provides powerful tools for visualizing and analyzing seismic ray propagation in 1D Earth models.